Есть пользователи, которые ставят друг другу лайки. Создать таблицы для хранения инфы. Написать запрос, который выведет id пользователя, имя, лайков получено, лайков поставлено, взаимных лайков. Написать запрос, который посчитает 5 самых популярных пользователей

In [1]:
import psycopg2 as pg_driver

db = pg_driver.connect(
    database="testdb", 
    user='testuser',
    password='testuser', 
    host='localhost', 
    port='5432'
)


def execute_queries(db, sql_commands):
    db.autocommit = True
    with db.cursor() as cursor:
        for sql_command in sql_commands:
            print(sql_command)
            cursor.execute(sql_command)
                     
sql_commands = ["DROP TABLE IF EXISTS users;",
                "DROP TABLE IF EXISTS likes;",
                """CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                """,
                """CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                """]


execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS likes;
CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                
CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                


In [2]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                     database="testdb", 
                     user='testuser',
                     password='testuser', 
                     host='localhost', 
                     port='5432'
                 );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

row_count_hist = execute_query("select count(*) from users", fetch_result=True)
row_count_payment = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)

[Record(count=0)]
[Record(count=0)]


In [3]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Mark'),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Dima'),
                 (3, to_timestamp('16-07-2020 09:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Phill'),
                 (4, to_timestamp('16-06-2018 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Viktor'),
                 (5, to_timestamp('16-06-2018 16:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Ivan')
                 
            
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (3, to_timestamp('16-07-2020 09:36:38', 'dd-mm-yyyy hh24:mi:ss'), 3),
                 (4, to_timestamp('16-06-2018 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 2),
                 (5, to_timestamp('16-06-2018 16:36:38', 'dd-mm-yyyy hh24:mi:ss'), 1)
                 
            
        """

execute_query(query)

In [4]:
query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (1, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5)
                 

                 
            
        """

execute_query(query)

In [5]:
all_rows = execute_query("select user_id, count(*) from likes group by user_id", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=3, count=1)
1 Record(user_id=5, count=1)
2 Record(user_id=4, count=1)
3 Record(user_id=2, count=2)
4 Record(user_id=1, count=3)


In [6]:
query = """select
                l.user_id as from_id,
                u.name as from_name,
                l.user_id_give as to_id,
                us.name as to_name
           from likes l
           left join users u
           on l.user_id=u.user_id
           left join users us
           on l.user_id_give=us.user_id
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(from_id=5, from_name='Ivan', to_id=1, to_name='Mark')
1 Record(from_id=4, from_name='Viktor', to_id=2, to_name='Dima')
2 Record(from_id=3, from_name='Phill', to_id=3, to_name='Phill')
3 Record(from_id=2, from_name='Dima', to_id=4, to_name='Viktor')
4 Record(from_id=1, from_name='Mark', to_id=4, to_name='Viktor')
5 Record(from_id=1, from_name='Mark', to_id=5, to_name='Ivan')
6 Record(from_id=2, from_name='Dima', to_id=5, to_name='Ivan')
7 Record(from_id=1, from_name='Mark', to_id=5, to_name='Ivan')


In [7]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (10, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Mark')
 
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (10, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4)
            
        """

execute_query(query)

In [9]:
query = """select
                --l.user_id as from_id,
                u.name as from_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id=u.user_id
           group by u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(from_name='Viktor', like_count=1)
1 Record(from_name='Phill', like_count=1)
2 Record(from_name='Ivan', like_count=1)
3 Record(from_name='Mark', like_count=4)
4 Record(from_name='Dima', like_count=2)


In [10]:
query = """select
                l.user_id_give as to_id,
                u.name as to_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id_give=u.user_id
           group by l.user_id_give, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(to_id=4, to_name='Viktor', like_count=3)
1 Record(to_id=1, to_name='Mark', like_count=1)
2 Record(to_id=3, to_name='Phill', like_count=1)
3 Record(to_id=2, to_name='Dima', like_count=1)
4 Record(to_id=5, to_name='Ivan', like_count=3)


In [11]:
query = """select t1.user_id, t1.user_id_give,
           count(*) as like_count
                
           from likes t1
           inner join likes t2
           on t1.user_id_give=t2.user_id
           where t2.user_id_give=t1.user_id 
           group by t1.user_id_give, t1.user_id
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=5, user_id_give=1, like_count=2)
1 Record(user_id=4, user_id_give=2, like_count=1)
2 Record(user_id=3, user_id_give=3, like_count=1)
3 Record(user_id=2, user_id_give=4, like_count=1)
4 Record(user_id=1, user_id_give=5, like_count=2)


In [12]:
query = """select
                l.user_id_give as to_id,
                u.name as to_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id_give=u.user_id
           group by l.user_id_give, u.name
           order by l.user_id_give DESC
           limit 5
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)   

0 Record(to_id=5, to_name='Ivan', like_count=3)
1 Record(to_id=4, to_name='Viktor', like_count=3)
2 Record(to_id=3, to_name='Phill', like_count=1)
3 Record(to_id=2, to_name='Dima', like_count=1)
4 Record(to_id=1, to_name='Mark', like_count=1)


В воображаемой социальной сети есть Пользователи (айди, имя), Фото (айди, название, автор) и Комментарии к фото (айди, текст, автор, айди фото). Необходимо добавить возможность для пользователей ставить лайки другим пользователям, фото или комментариям к фото. Нужно реализовать такие возможности:
пользователь имеет право отозвать лайк;
необходимо иметь возможность посчитать число полученных сущностью лайков и вывести список пользователей, поставивших лайки;
в будущем могут появиться новые виды сущностей, которые можно лайкать.

In [13]:
import psycopg2 as pg_driver

db = pg_driver.connect(
    database="testdb", 
    user='testuser',
    password='testuser', 
    host='localhost', 
    port='5432'
)


def execute_queries(db, sql_commands):
    db.autocommit = True
    # тот же самый курсор
    with db.cursor() as cursor:
        # в цикле поочередно выполняем запросы
        for sql_command in sql_commands:
            print(sql_command)
            # тот же самый .execute() для выполнения запроса
            cursor.execute(sql_command)
            
sql_commands = ["DROP TABLE IF EXISTS users2;",
                "DROP TABLE IF EXISTS photos;",
                "DROP TABLE IF EXISTS comments;"
                "DROP TABLE IF EXISTS likes;",
                """CREATE TABLE users2 (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL
                );
                """,
                """CREATE TABLE photos (
                         photo_id    INT       NOT NULL,
                         photo_name    TEXT       NOT NULL,
                         author_id    INT       NOT NULL
                  );
                """,
               """CREATE TABLE comments (
                         comment_id    INT       NOT NULL,
                         user_id    INT       NOT NULL,
                         text    TEXT       NOT NULL,
                         author_id    INT       NOT NULL,
                         photo_id    INT       NOT NULL
                );
                """,
               """CREATE TABLE likes (
                         user_from    INT       NOT NULL,
                         entity_type       TEXT       NOT NULL,
                         entity_id    INT       NOT NULL,
                         user_to     INT       NOT NULL    
                         
                );
                """]


execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users2;
DROP TABLE IF EXISTS photos;
DROP TABLE IF EXISTS comments;DROP TABLE IF EXISTS likes;
CREATE TABLE users2 (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL
                );
                
CREATE TABLE photos (
                         photo_id    INT       NOT NULL,
                         photo_name    TEXT       NOT NULL,
                         author_id    INT       NOT NULL
                  );
                
CREATE TABLE comments (
                         comment_id    INT       NOT NULL,
                         user_id    INT       NOT NULL,
                         text    TEXT       NOT NULL,
                         author_id    INT       NOT NULL,
                         photo_id    INT       NOT NULL
                );
                
CREATE TABLE likes (
                         user_from    INT       NOT NULL,
                         entity_type       TEXT       NOT NUL

In [14]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                     database="testdb", 
                     user='testuser',
                     password='testuser', 
                     host='localhost', 
                     port='5432'
                 );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

row_count_hist = execute_query("select count(*) from users2", fetch_result=True)
row_count_payment = execute_query("select count(*) from photos", fetch_result=True)
row_count_comm = execute_query("select count(*) from comments", fetch_result=True)
row_count_lik = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)
print(row_count_comm)
print(row_count_lik)

[Record(count=0)]
[Record(count=0)]
[Record(count=0)]
[Record(count=0)]


In [15]:
query = """ INSERT INTO users2 (user_id, name) 
            VALUES 
                 (1, 'Mark'),
                 (2, 'Dima'),
                 (3, 'Pina'),
                 (4, 'Alice'),
                 (5, 'Rob'),
                 (6, 'Ann'),
                 (7, 'Jin')
                 
            
        """

execute_query(query)

query = """ INSERT INTO photos (photo_id, photo_name, author_id ) 
            VALUES 
                 (1, 'Forest', 5),
                 (2, 'River', 4),
                 (3, 'City', 3),
                 (4, 'Portret', 7),
                 (5, 'Street', 1),
                 (6, 'Member', 7),
                 (7, 'Home', 4),
                 (8, 'Peach', 6),
                 (9, 'Flowers', 4),
                 (10, 'Cats', 2),
                 (11, 'Dogs', 2)
                 
            
        """

execute_query(query)

query = """ INSERT INTO comments (comment_id, user_id, text, author_id, photo_id ) 
            VALUES 
                 (1, 1, 'Cool', 6, 8),
                 (2, 7, 'Cute', 2, 10),
                 (3, 4, 'Want to be there', 5, 1),
                 (4, 4, 'WWH', 7, 4),
                 (5, 5, 'Beautiful', 4, 9)
                 
            
        """

execute_query(query)

query = """ INSERT INTO likes (user_from, entity_type, entity_id, user_to) 
            VALUES 
                 (1, 'person', 3, 3),
                 (2, 'photo', 5, 1),
                 (3, 'comment', 3, 4),
                 (4, 'photo', 8, 6),
                 (5, 'person', 7, 7)
            
        """

execute_query(query)

In [16]:
all_rows = execute_query("select * from users2", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, name='Mark')
1 Record(user_id=2, name='Dima')
2 Record(user_id=3, name='Pina')
3 Record(user_id=4, name='Alice')
4 Record(user_id=5, name='Rob')
5 Record(user_id=6, name='Ann')
6 Record(user_id=7, name='Jin')


In [17]:
query = """select
                u.name,
                --p.photo_name,
                count(*) as photo_count
                
           from users2 u
           left join photos p
           on u.user_id=p.author_id
           group by u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(name='Pina', photo_count=1)
1 Record(name='Mark', photo_count=1)
2 Record(name='Ann', photo_count=1)
3 Record(name='Alice', photo_count=3)
4 Record(name='Jin', photo_count=2)
5 Record(name='Rob', photo_count=1)
6 Record(name='Dima', photo_count=2)


In [18]:
query = """select
                l.user_from,
                u.name,
                count(*) as like_count
                
           from likes l
           left join users2 u
           on l.user_from=u.user_id
           group by l.user_from, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_from=1, name='Mark', like_count=1)
1 Record(user_from=3, name='Pina', like_count=1)
2 Record(user_from=2, name='Dima', like_count=1)
3 Record(user_from=5, name='Rob', like_count=1)
4 Record(user_from=4, name='Alice', like_count=1)


In [19]:
query = """select
                l.user_to,
                u.name,
                count(*) as like_count
                
           from likes l
           left join users2 u
           on l.user_to=u.user_id
           group by l.user_to, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_to=1, name='Mark', like_count=1)
1 Record(user_to=7, name='Jin', like_count=1)
2 Record(user_to=3, name='Pina', like_count=1)
3 Record(user_to=4, name='Alice', like_count=1)
4 Record(user_to=6, name='Ann', like_count=1)
